# 1. Question

In [1]:
import requests
rss_url = "http://fs.jtbc.joins.com/RSS/economy.xml"
jtbc_economy = requests.get(rss_url)

In [2]:
from bs4 import BeautifulSoup
economy_news_list = BeautifulSoup(jtbc_economy.content, "xml")
link_list = economy_news_list.select("item > link")

In [3]:
from konlpy.tag import Mecab
mecab = Mecab()

In [5]:
news =[]
for link in link_list:
    news_url = link.text
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.content, "html.parser")
    news_content = news_soup.select_one("#articlebody > .article_content")
    news.append(list(filter(lambda word: len(word)>1, mecab.nouns(news_content.text))))
    
news

[['아파트',
  '가구',
  '분양',
  '민간',
  '아파트',
  '분양',
  '상한',
  '적용',
  '건설사',
  '분양',
  '기록',
  '청약',
  '경쟁',
  '이달',
  '다음',
  '물량',
  '가구',
  '정부',
  '발표',
  '가구',
  '서울',
  '가구',
  '가구',
  '코리아',
  '벌금',
  '확정',
  '배출',
  '가스',
  '인증',
  '서류',
  '위조',
  '시험',
  '적서',
  '조작',
  '국내',
  '차량',
  '혐의',
  '벤츠',
  '코리아',
  '벌금',
  '확정',
  '소식',
  '코리아',
  '벌금',
  '확정',
  '담당',
  '직원',
  '서류',
  '위조',
  '벤츠',
  '코리아',
  '의도',
  '판결',
  '법원',
  '아시아',
  '인수',
  '후보',
  '압축',
  '아시아',
  '항공',
  '예비',
  '입찰',
  '참여',
  '기업',
  '가운데',
  '적격',
  '인수',
  '후보',
  '애경그룹',
  '포함',
  '통보',
  '재무',
  '투자자',
  '아시아나항공',
  '실사',
  '참여',
  '다음',
  '입찰',
  '진행',
  '올해',
  '매각',
  '작업',
  '마무리',
  '예정'],
 ['앵커',
  '내년',
  '인구',
  '주택',
  '총조사',
  '반려',
  '동물',
  '항목',
  '방안',
  '정부',
  '검토',
  '여름',
  '처음',
  '시험',
  '조사',
  '내년',
  '확정',
  '계획',
  '반려',
  '동물',
  '가족',
  '여기',
  '사회',
  '분위기',
  '반영',
  '송지혜',
  '기자',
  '기자',
  '통계청',
  '가구',
  '상대',
  '실시',
  '인구',
  '주택',
  '시험',
  '조사',
  '항목',
  '반려',
 

# 2. Question

In [6]:
word_Korean = ["추석", "연휴", "민족", "대이동", "시작", "늘어", "교통량", "교통사고", "특히", "자동차", \
               "고장", "상당수", "차지", "나타", "것", "기자"]

In [7]:
stopwords = ["가다", "늘어", "나타", "것", "기자"] 

In [8]:
results = [word for word in word_Korean if word not in stopwords]
results

['추석', '연휴', '민족', '대이동', '시작', '교통량', '교통사고', '특히', '자동차', '고장', '상당수', '차지']

# 3. Question

In [37]:
pip install pymongo

     |████████████████████████████████| 450kB 214kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install selenium

     |████████████████████████████████| 911kB 586kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [49]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
import re
import pandas as pd

#명시적 대기를 위해서
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import time

title_list = list()
point_list = list()
regdate_list = list()
content_list = list()


for i in range(1, 2):
    driver = wd.Chrome(executable_path = './chromedriver')
    url = 'http://www.yes24.com/Product/communityModules/GoodsReviewList/40936880?Sort=1&PageNumber='
    url += str(i) + '&Type=ALL&_=156817'
    driver.get(url)

    # #암묵적 대기
    driver.implicitly_wait(2)
    #제목, 평점, 작성날짜, 리뷰내용
    #review_Tit,review_rating, txt_date, review_cont
    time.sleep(2)
    review_info_group = driver.find_elements_by_css_selector('.reviewInfoGrp')
    for review_info in review_info_group:
        title = review_info.find_element_by_css_selector('.review_Tit').text
        review_rating = review_info.find_element_by_css_selector('.review_rating').text  #내용 평점5점   편집/디자인 평점5점
        txt_date = review_info.find_element_by_css_selector('.txt_date').text
        review_cont = review_info.find_element_by_css_selector('.review_cont').text
        title_list.append(title)
        point = re.sub(r'평점', '', review_rating.split(' ')[1])
        point_list.append(re.sub(r'점', '', point))
        regdate_list.append(txt_date)
        content_list.append(review_cont)
    driver.close()

df = pd.DataFrame(title_list, columns = ['title'])
df['point'] = point_list
df['regdate'] = regdate_list
df['review'] = content_list

import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client.test

try:
    for i in range(0, len(title_list)):
        db.insight.insert({
            'title' : df.loc[i, 'title'],
            'point' : df.loc[i, 'point'],
            'regdate' : df.loc[i, 'regdate'],
            'review' : df.loc[i, 'review']
        })
    print("Success.")
except Exception as e: 
    print('Exception 발생 = ', e)


/home/instructor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:59: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


Exception 발생 =  localhost:27017: [Errno 111] Connection refused


In [50]:
review_list = list()
reviews = db.insight.find({}, {'review':1, '_id':0})
for review in reviews:
    review_list.append(review)
    
print(review_list)

ServerSelectionTimeoutError: localhost:27017: [Errno 111] Connection refused

In [ ]:
text = ''
reviews = db.insight.find({}, {'review':1, '_id':0})
for review in reviews:
    text += review['review']

text
from konlpy.tag import Kkma
kkma = Kkma()
nouns = kkma.nouns(text)
nouns_list = [noun for noun in nouns if len(noun) > 2]
text = ' '.join(nouns_list)

In [ ]:
cloud = WordCloud(background_color = 'white', \
                  font_path = 'C:/Windows/Fonts/malgun.ttf')

In [ ]:
cloud.generate(text)

In [ ]:
plt.figure(figsize=(15, 10))
plt.imshow(cloud, interpolation = 'bilinear')
plt.axis('off')